In [1]:
import re
import requests
import os
from datetime import datetime

from bs4 import BeautifulSoup, Comment
from bs4.element import Tag

import pandas as pd
from tqdm import tqdm

In [2]:
def get_description(breed_soup):
    try:
        description = breed_soup.find(
                'p', class_='breed-page__about__read-more__text'
        ).get_text().strip()
    except:
        description = ''
        
    
    # Removing weird characters
    # Probably not exhaustive
    description = description.replace(
        '\n', '').replace('\u200b', '').replace('\xa0', ' ')
    
    return description

In [3]:
def get_temperament(breed_soup):
    temperment_class = 'breed-page__intro__temperment'
    try:
        return breed_soup.find(
            'p', class_=temperment_class
        ).get_text().replace(' / ', ',')
    except:
        return ''

In [4]:
def general_regex(text, var, mul=1):
    reg = re.compile('(\d+\.?\d*)')
    results = reg.findall(text)
    numbers = [float(value) * mul for value in results]
    if len(numbers) == 1:
        numbers = numbers * 2
    elif len(numbers) == 0:
        numbers = [0, 0]
    return {
        'min_{}'.format(var): min(numbers),
        'max_{}'.format(var): max(numbers)
    }

In [5]:
# Define scraping functions for each attribute
def get_popularity(attribute):
    return attribute.replace('Ranks', '').split('of')[0].strip().split(':')[1]

def get_height(attribute):
    return general_regex(attribute, 'height')

def get_weight(attribute):
    return general_regex(attribute, 'weight') 

def get_expectancy(attribute):
    return general_regex(attribute, 'expectancy')

def get_group(attribute):
    return attribute.split(':')[1].strip()

# Create a dictionary mapping attribute names to their respective scraping functions
attr_function = {
    'akc_breed_popularity': get_popularity,
    'height': get_height,
    'weight': get_weight,
    'life_expectancy': get_expectancy,
    'group': get_group
}

# Function to extract attributes from a comment
def get_attributes_from_comment(comment):
    attributes_data = {}
    
    comment_content = BeautifulSoup(comment, 'html.parser')
    
    data_objects = comment_content.find_all('dataobject', type='document')
    
    for data_object in data_objects:
        attributes = data_object.find_all('attribute')
        
        for attribute in attributes:
            name = attribute.get('name')
            value = attribute.get_text(strip=True)
            
            # Check if the attribute name is in the attr_function dictionary
            if name in attr_function:
                scraping_function = attr_function[name]
                if(name in ['height', 'weight', 'life_expectancy']):
                    attr_dict = scraping_function(value)
                    for key in attr_dict.keys():
                        attributes_data[key] = attr_dict[key]
                    
                else:
                    attributes_data[name] = scraping_function(value)
    
    return attributes_data

In [6]:
def get_main_attributes(breed_soup):
    breed_attributes = []
    # Find all comments in the HTML
    comments = breed_soup.find_all(string=lambda text: isinstance(text, Comment))

    # Iterate through comments
    for comment in comments:
        comment_content = BeautifulSoup(comment, 'html.parser')

        if comment_content.find('pagemap'):
            breed_attributes = get_attributes_from_comment(comment)
    
    print('breed_attributes', breed_attributes)
    return breed_attributes

In [7]:
def get_care_info(breed_soup):
    titles = breed_soup.find_all(
        'h4', class_='bar-graph__title'
    )
    
    values = breed_soup.find_all(
        'div', class_='bar-graph__section'
    )
    
    categories = breed_soup.find_all(
        'div', class_='bar-graph__text'
    )
    
    care_dict = {}
    
    for (title, value, category) in zip (titles, values, categories):
        t = title.get_text().lower().replace(' ', '_')
        t = t[t.find('/') + 1:]
        care_dict[t + '_value'] = float(
            value['style'].split()[1].split('%')[0]
        ) / 100
        care_dict[t + '_category'] = category.get_text()
        
    return care_dict

In [8]:
# Working
def get_breed_characteristics(breed_soup):
    breed_characteristics = {}
    characteristic_classes = [tab_element.getText() for tab_element in breed_soup.find_all('li', class_='tabs__single-tab')]
    for characteristic_class in characteristic_classes:
        if('Traits' not in characteristic_class):
            characteristic_class_name = f'breed-page__traits__{characteristic_class.lower().replace(" ", "-")}'
            characteristics_div = breed_soup.find('div', id=characteristic_class_name)
            breed_traits = characteristics_div.find_all('div', class_='breed-trait-group__trait')
            breed_group_characteristics = {}
            for breed_trait in breed_traits:
                trait = breed_trait.find('h4', class_='accordion__header__text').getText()
                if trait in ['Coat Type', 'Coat Length']:
                    coatParentDivs = breed_trait.find_all(class_='breed-trait-score__choice--selected')
                    coatNames = []
                    for coatParentDiv in coatParentDivs:
                        span_text = coatParentDiv.find('span').text
                        coatNames.append(span_text)
                    trait_info = ",".join(coatNames)
                else:
                    trait_info = len(breed_trait.find_all('div', class_='breed-trait-score__score-unit breed-trait-score__score-unit--filled'))
                breed_group_characteristics[trait] = trait_info
                breed_characteristics[trait] = trait_info
    return breed_characteristics

In [9]:
# Images
def get_breed_images(breed_soup):
    # Extract og:image URL
        og_image_tag = breed_soup.find('meta', attrs={'property': 'og:image'})
        if og_image_tag:
            return {'images': [og_image_tag['content']]}
        else:
            return {'images': []}

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

class Breed:
    def __init__(self, breed_url, breed_name):
        self.url = breed_url
        self.name = breed_name
        
        # Set up a headless Chrome browser
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Run in headless mode (no GUI)
        driver = webdriver.Chrome(options=options)

        # Load the web page
        driver.get(breed_url)

        # Get the page source after content has loaded
        page_source = driver.page_source

        # Create a BeautifulSoup object from the page source
        breed_soup = BeautifulSoup(page_source, 'html.parser')

        # Scrape content as needed
        self.breed_info = {}
        self.breed_info['breed'] = self.name
        self.breed_info['description'] = get_description(breed_soup)
        self.breed_info['temperament'] = get_temperament(breed_soup)
        self.breed_info.update(get_main_attributes(breed_soup))
        self.breed_info.update(get_care_info(breed_soup))
        self.breed_info.update(get_breed_characteristics(breed_soup))
        self.breed_info.update(get_breed_images(breed_soup))

        # Close the browser
        driver.quit()

    def get_breed_info(self):
        return self.breed_info


In [11]:
import requests
from bs4 import BeautifulSoup
from bs4 import Tag
from tqdm import tqdm  # Import tqdm for the progress bar

def get_data():
    page = requests.get('https://www.akc.org/dog-breeds/')
    soup = BeautifulSoup(page.content, 'html.parser')
    
    breed_select = soup.find('select', id='breed-search')

    breeds = []
    
    for tag in breed_select.children:
        if isinstance(tag, Tag):
            if 'value' in tag.attrs and tag['value']:
                breeds.append(tag)

    print(len(breeds))
                
    breed_dict = {}
    
    # Use tqdm to create a progress bar
    for breed in tqdm(breeds[250:],desc='scraping Breeds'):
        breed_name = breed.get_text()
        breed_url = breed['value']
        breed_info = Breed(breed_url, breed_name).get_breed_info()
        breed_dict[breed_name] = breed_info
    
#     print('breed_dict', breed_dict)
    
    return breed_dict

# Define the Breed class and its methods here if not already defined

# Call get_data() to start the scraping process

In [12]:
breed_dict = get_data()

288


scraping Breeds:   0%|                                                                          | 0/38 [00:00<?, ?it/s]

breed_attributes {'akc_breed_popularity': '  66', 'min_height': 17.0, 'max_height': 19.0, 'min_weight': 30.0, 'max_weight': 40.0, 'min_expectancy': 12.0, 'max_expectancy': 14.0, 'group': 'Terrier Group'}


scraping Breeds:   3%|█▋                                                                | 1/38 [00:13<08:30, 13.80s/it]

breed_attributes {'akc_breed_popularity': '', 'min_height': 28.0, 'max_height': 35.0, 'min_weight': 140.0, 'max_weight': 200.0, 'min_expectancy': 10.0, 'max_expectancy': 12.0, 'group': 'Foundation Stock Service'}


...

scraping Breeds:  95%|█████████████████████████████████████████████████████████████▌   | 36/38 [06:36<00:21, 10.53s/it]

breed_attributes {'akc_breed_popularity': '', 'min_height': 21.0, 'max_height': 23.0, 'min_weight': 40.0, 'max_weight': 55.0, 'min_expectancy': 10.0, 'max_expectancy': 12.0, 'group': 'Foundation Stock Service'}


scraping Breeds:  97%|███████████████████████████████████████████████████████████████▎ | 37/38 [06:46<00:10, 10.41s/it]

breed_attributes {'akc_breed_popularity': '  13', 'min_height': 7.0, 'max_height': 8.0, 'min_weight': 7.0, 'max_weight': 7.0, 'min_expectancy': 11.0, 'max_expectancy': 15.0, 'group': 'Toy Group'}


scraping Breeds: 100%|█████████████████████████████████████████████████████████████████| 38/38 [06:57<00:00, 10.99s/it]


In [13]:
breed_df = pd.DataFrame.from_dict(breed_dict, orient='index')

In [14]:
breed_df

,breed,description,temperament,akc_breed_popularity,min_height,max_height,min_weight,max_weight,min_expectancy,max_expectancy,...,Coat Length,Openness To Strangers,Playfulness Level,Watchdog/Protective Nature,Adaptability Level,Trainability Level,Energy Level,Barking Level,Mental Stimulation Needs,images
Soft Coated Wheaten Terrier,Soft Coated Wheaten Terrier,"The Soft Coated Wheaten Terrier, an exuberant ...","friendly,happy,deeply devoted",66,17.00,19.00,30.0,40.0,12.0,14.0,...,Medium,3,3,3,3,3,4,3,3,[https://www.akc.org/wp-content/uploads/2017/1...
Spanish Mastiff,Spanish Mastiff,"The Spanish Mastiff is a very intelligent dog,...","intelligent,affectionate,kind",,28.00,35.00,140.0,200.0,10.0,12.0,...,Short,3,2,5,3,3,3,1,3,[https://www.akc.org/wp-content/uploads/2017/1...
Spanish Water Dog,Spanish Water Dog,The inexhaustible Spanish Water Dog is a dual-...,"Upbeat,Playful but also work-oriented,Very Active",163,15.75,19.75,31.0,49.0,12.0,14.0,...,Medium,3,4,3,3,4,4,3,5,[https://www.akc.org/wp-content/uploads/2017/1...
Spinone Italiano,Spinone Italiano,"The Spinone Italiano, a densely-coated hunting...","sociable,patient,docile",112,22.00,27.00,0.0,0.0,10.0,12.0,...,Medium,3,3,3,3,4,3,2,3,[https://www.akc.org/wp-content/uploads/2017/1...
Stabyhoun,Stabyhoun,Although occasionally somewhat willful by natu...,"intelligent,loving,responsive",,19.00,21.00,40.0,60.0,13.0,15.0,...,Short,3,3,3,3,5,4,3,4,[https://www.akc.org/wp-content/uploads/2017/1...
Staffordshire Bull Terrier,Staffordshire Bull Terrier,"From his brawling past, the muscular but agile...","clever,brave,tenacious",75,14.00,16.00,24.0,38.0,12.0,14.0,...,Short,4,4,5,5,5,4,3,4,[https://www.akc.org/wp-content/uploads/2017/1...
Standard Schnauzer,Standard Schnauzer,"The bold, bewhiskered Standard Schnauzer is a ...","fearless,smart,spirited",94,17.50,19.50,30.0,50.0,13.0,16.0,...,Medium,3,4,5,4,5,3,3,5,[https://www.akc.org/wp-content/uploads/2017/1...
Sussex Spaniel,Sussex Spaniel,"""Placid, affectionate, even-tempered, true-blu...","friendly,merry,even-tempered",192,13.00,15.00,35.0,45.0,13.0,15.0,...,Medium,3,4,3,4,4,3,3,3,[https://www.akc.org/wp-content/uploads/2017/1...
Swedish Lapphund,Swedish Lapphund,"Lively, alert, kind and affectionate. The Lapp...","intelligent,playful,lively",,16.00,20.00,30.0,45.0,12.0,14.0,...,Long,4,4,4,4,5,4,4,4,[https://www.akc.org/wp-content/uploads/2017/1...
Swedish Vallhund,Swedish Vallhund,"The long and low Swedish Vallhund, Viking Dog ...","friendly,energetic,watchful",160,11.50,13.75,20.0,35.0,12.0,15.0,...,Short,3,4,3,4,4,4,3,4,[https://www.akc.org/wp-content/uploads/2017/1...


In [15]:
excel_file_path = 'to_excel.xlsx'

In [16]:
breed_df.to_excel(excel_file_path, index=False)